#### Create graph with three nodes

```
CREATE (t:Teacher{name: 'Ms. Higgins'})
CREATE (p1:Pupil{name: 'Mike', born: 'March 27, 2010'})
CREATE (p2:Pupil{name: 'Mary', born: 'January 7, 2010'})
```

#### Create Teacher's Pet Relationship

```
MATCH (p:Pupil), (t:Teacher)
WHERE p.name = 'Mary' AND t.name = 'Ms. Higgins'
CREATE (p)-[tp:TEACHERS_PET_OF]->(t)
```

#### Create Relationship bt Teacher and Students

```
MATCH (p:Pupil), (t:Teacher)
CREATE (t)-[e:EDUCATES]->(p)
```

#### Return Graph

``` MATCH (p:Pupil), (t:Teacher)
RETURN p, t```

#### Query for students who are teachers pets

```
MATCH (p)-[tp:TEACHERS_PET_OF]->(t)
RETURN p
```

#### Delete students who are not teacher's pets

```
MATCH (t:Teacher)-[r:EDUCATES]-(p:Pupil)
WHERE p.name = 'Mike'
DELETE r
```
```
MATCH (p:Pupil)
WHERE NOT (p)-[:TEACHERS_PET_OF]->()
DELETE p
```


![title](remaining-school.png)


![title](movie-db.png)

After loading the attached file "create_movies_db.cypher" I was able to determine that there are 253 relationships and 6 types. Below is a screenshot detailing the different relationships.

![title](relationships-movie.png)

### Confirm number of relationships for node with several relationships

```
MATCH (p:Person {name:'Tom Hanks'})-[r]-()
RETURN r, size((p)-->()) AS count
```

### Select relationships for movie label for each node in descending order

```
MATCH (m:Movie)-[r]-(b)
RETURN m.title AS Title, count(r) AS count
ORDER by count DESC
```

In [5]:
#Add John Wick movie, co-stars and directors to graph db

from neo4j import GraphDatabase

neo_conn = GraphDatabase.driver(uri = 'bolt://localhost:7687', auth = ('neo4j', 'test'))
neo_session = neo_conn.session()

# execute query
query = '''
CREATE (m:Movie { title:"John Wick",released:2014 })
CREATE (keanu:Person { name:"Keanu Reeves", born:1964 })
CREATE (keanu)-[:ACTED_IN { roles: ['John Wick']}] ->(m)
CREATE (d1:Person { name:"Chad Stahelski"})-[di1:DIRECTED]->(m)
CREATE (d2:Person { name:"David Leitch"})-[di2:DIRECTED]->(m)
CREATE (a1:Person { name:"William Dafoe"})
CREATE (a1)-[:ACTED_IN { roles: ['John Wick']}] ->(m)
CREATE (a2:Person { name:"Michael Nyquist"})
CREATE (a2)-[:ACTED_IN { roles: ['John Wick']}] ->(m)
RETURN m,keanu,a1,a2, d1, d2;
'''
executed_query = neo_session.run(query)

# convert to list for printing
result = list(executed_query)
print(result)

[<Record m=<Node id=187 labels=frozenset({'Movie'}) properties={'title': 'John Wick', 'released': 2014}> keanu=<Node id=188 labels=frozenset({'Person'}) properties={'born': 1964, 'name': 'Keanu Reeves'}> a1=<Node id=191 labels=frozenset({'Person'}) properties={'name': 'William Dafoe'}> a2=<Node id=192 labels=frozenset({'Person'}) properties={'name': 'Michael Nyquist'}> d1=<Node id=189 labels=frozenset({'Person'}) properties={'name': 'Chad Stahelski'}> d2=<Node id=190 labels=frozenset({'Person'}) properties={'name': 'David Leitch'}>>]


### Query for all Keanu Reeves movies and co-stars

```
MATCH (m:Movie)<-[:ACTED_IN]-(p:Person)
RETURN m.title AS movie, collect(DISTINCT p.name) AS cast, count(*) AS 
actors;
```

![title](text-report.png)

#### Create list of actors playing in movies which Keanu Reeves played

In [7]:
# execute query
query = '''
MATCH (p:Person)
WHERE (:Person{name: 'Keanu Reeves'})-[:ACTED_IN]->(:Movie)
RETURN DISTINCT p
'''
executed_query = neo_session.run(query)

# convert list for printing
result = list(executed_query)
print(result)

[<Record p=<Node id=0 labels=frozenset({'Person'}) properties={'born': 1964, 'name': 'Keanu Reeves'}>>, <Record p=<Node id=2 labels=frozenset({'Person'}) properties={'born': 1967, 'name': 'Carrie-Anne Moss'}>>, <Record p=<Node id=3 labels=frozenset({'Person'}) properties={'born': 1961, 'name': 'Laurence Fishburne'}>>, <Record p=<Node id=4 labels=frozenset({'Person'}) properties={'born': 1960, 'name': 'Hugo Weaving'}>>, <Record p=<Node id=5 labels=frozenset({'Person'}) properties={'born': 1967, 'name': 'Lilly Wachowski'}>>, <Record p=<Node id=6 labels=frozenset({'Person'}) properties={'born': 1965, 'name': 'Lana Wachowski'}>>, <Record p=<Node id=7 labels=frozenset({'Person'}) properties={'born': 1952, 'name': 'Joel Silver'}>>, <Record p=<Node id=8 labels=frozenset({'Person'}) properties={'born': 1978, 'name': 'Emil Eifrem'}>>, <Record p=<Node id=12 labels=frozenset({'Person'}) properties={'born': 1975, 'name': 'Charlize Theron'}>>, <Record p=<Node id=13 labels=frozenset({'Person'}) prop

### Find directors of movies in which Keanu Reeves starred in

In [8]:
# execute query
query = '''
MATCH (p:Person), (m:Movie)
WHERE (:Person{name: 'Keanu Reeves'})-[:ACTED_IN]->(m:Movie)
AND (p)-[:DIRECTED]->(m)
RETURN COLLECT(DISTINCT p)
'''
executed_query = neo_session.run(query)

# convert list for printing
result = list(executed_query)
print(result)

[<Record COLLECT(DISTINCT p)=[<Node id=5 labels=frozenset({'Person'}) properties={'born': 1967, 'name': 'Lilly Wachowski'}>, <Node id=6 labels=frozenset({'Person'}) properties={'born': 1965, 'name': 'Lana Wachowski'}>, <Node id=14 labels=frozenset({'Person'}) properties={'born': 1944, 'name': 'Taylor Hackford'}>, <Node id=91 labels=frozenset({'Person'}) properties={'born': 1950, 'name': 'Howard Deutch'}>, <Node id=104 labels=frozenset({'Person'}) properties={'born': 1953, 'name': 'Robert Longo'}>, <Node id=156 labels=frozenset({'Person'}) properties={'born': 1949, 'name': 'Nancy Meyers'}>, <Node id=178 labels=frozenset({'Person'}) properties={'name': 'David Leitch'}>, <Node id=183 labels=frozenset({'Person'}) properties={'name': 'Chad Stahelski'}>, <Node id=184 labels=frozenset({'Person'}) properties={'name': 'David Leitch'}>, <Node id=189 labels=frozenset({'Person'}) properties={'name': 'Chad Stahelski'}>, <Node id=190 labels=frozenset({'Person'}) properties={'name': 'David Leitch'}>]